## 2. NetCDF 데이터 불러오기

### 2.1 NetCDF란?

NetCDF(Network Common Data Form)는 기후/해양 데이터의 표준 포맷 
- **다차원 배열** 저장에 최적화 (시간, 위도, 경도, 깊이)
- **메타데이터** 포함 (단위, 설명, 좌표계 정보)
- **자기 기술적(self-describing)**: 파일 자체에 구조 정보가 담겨 있음

### 2.2 OISST 데이터 불러오기
OISST(Optimum Interpolation Sea Surface Temperature)는 NOAA에서 제공하는 일별 전지구 SST 데이터

In [4]:
import xarray as xr
import os
from pathlib import Path

In [5]:
import numpy as np
from datetime import datetime

In [9]:
import copernicusmarine as cm

In [6]:
# ===== ===== setting ===== =====
START_YEAR = 2015
END_YEAR = 2024

# kuroshio
LON_MIN, LON_MAX = 115, 160
LAT_MIN, LAT_MAX = 20, 50
SAVE_DIR = Path("../data/raw/oisst")
SAVE_DIR2 = Path("../data/raw/glorys")

- https://psl.noaa.gov/thredds/dodsC
- Datasets/noaa.oisst.v2.highres/sst.mon.mean.nc

In [ ]:
# ===== ===== OISST monthly data URL ===== =====
OISST_URL = "https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.oisst.v2.highres/sst.mon.mean.nc"

In [ ]:
# 셀 3: 다운로드 (단순화 버전)
print("서버 연결 중...")
ds = xr.open_dataset(OISST_URL)
print(f"연결 성공! 변수: {list(ds.data_vars)}")

print("\n영역 추출 중...")
ds_subset = ds.sel(
    time=slice(f"{START_YEAR}-01-01", f"{END_YEAR}-12-31"),
    lat=slice(LAT_MIN, LAT_MAX),
    lon=slice(LON_MIN, LON_MAX)
)
print(f"추출 완료: {ds_subset.dims}")

print("\n다운로드 중... (1-2분)")
ds_subset = ds_subset.load()
print(f"다운로드 완료!")

print("\n저장 중...")
save_path = SAVE_DIR / f"oisst_monthly_{START_YEAR}_{END_YEAR}_kuroshio.nc"
ds_subset.to_netcdf(save_path, engine='scipy')
print(f"저장 완료: {save_path}")

ds.close()

In [ ]:
def download_oisst():
    # save folder
    SAVE_DIR.mkdir(parents=True, exist_ok=True)
    
    try:
        ds = xr.open_dataset(OISST_URL)
        print(" connection complete ")
        
        # data information
        print(" data information ")
        print(f" - time range: {ds.time.values[0]} ~ {ds.time.values[-1]}")
        print(f" - lat range: {float(ds.lat.min())} ~ {float(ds.lat.max())}")
        print(f" - lon range: {float(ds.lon.min())} ~ {float(ds.lon.max())}")
        
        # select period and region
        print(f" - time period: {START_YEAR}-01 ~ {END_YEAR}-12")
        print(f" - region range: {LON_MIN}-{LON_MAX}E, {LAT_MIN}-{LAT_MAX}N")
        
        ds_subset = ds.sel(
            time = slice(f"{START_YEAR}-01-01", f"{END_YEAR}-12-31"),
            lat = slice(LAT_MIN, LAT_MAX),
            lon = slice(LON_MIN, LON_MAX) 
        )
        
        # DATA LOAD
        ds_subset = ds_subset.load()
        
        # SAVE
        save_path = SAVE_DIR / f"oisst_monthly_{START_YEAR}_{END_YEAR}_kuroshio.nc"
        
        # COMPRESSION
        # encoding = {'sst': {'zlib': True, 'complevel': 4}}
        ds_subset.to_netcdf(save_path, engine = 'scipy')
        
        # validation
        ds_check = xr.open_dataset(save_path)
        print(f"   - SST RANGE: {float(ds_check.sst.min()):.1f} ~ {float(ds_check.sst.max()):.1f} °C")
        ds_check.close()
        ds.close()
        
        return save_path
    except Exception as e:
        print(f"Error: {e}")
        return None
## if __name__ == "__main__": download_oisst()

In [ ]:
# 파일 다운로드 중 겹침 현상 해결용
'''
save_path = SAVE_DIR / f"oisst_monthly_{START_YEAR}_{END_YEAR}_kuroshio.nc"
if save_path.exists():
    os.remove(save_path)
    print(f"원래 있던 파일 삭제하기: {save_path}")
'''

In [ ]:
download_oisst()

In [ ]:
# 파일 확인
test_path = SAVE_DIR / f"oisst_monthly_{START_YEAR}_{END_YEAR}_kuroshio.nc"
print(f"파일 존재: {test_path.exists()}")

if test_path.exists():
    ds_test = xr.open_dataset(test_path)
    print(ds_test)
    print(f"\nSST 범위: {float(ds_test.sst.min()):.1f} ~ {float(ds_test.sst.max()):.1f} °C")
    ds_test.close()

## 2.5 GLORYS 데이터 불러오기

- 기간, 영역은 oisst와 동일
- 변수는 MLD

In [7]:
# ===== ===== HYCOM monthly data URL ===== =====
GLORYS_URL = "https://my.cmems-du.eu/thredds/dodsC/cmems_mod_glo_phy_my_0.083deg_P1M-m"

In [ ]:
'''

import sys
print(sys.executable)

!pip install copernicusmarine
'''

In [1]:
'''

import copernicusmarine as cm

cm.login()
'''

'\n\nimport copernicusmarine as cm\n\ncm.login()\n'

In [10]:
SAVE_DIR2.mkdir(parents=True, exist_ok=True)

ds = cm.open_dataset(
    dataset_id = "cmems_mod_glo_phy_my_0.083deg_P1M-m",
    variables = ["mlotst"],
    minimum_longitude=115,
    maximum_longitude=160,
    minimum_latitude=20,
    maximum_latitude=50,
    start_datetime = "2015-01-01",
    end_datetime = "2024-12-31"
)

print(ds)

INFO - 2026-01-20T07:14:04Z - Selected dataset version: "202311"
INFO - 2026-01-20T07:14:04Z - Selected dataset part: "default"


<xarray.Dataset> Size: 187MB
Dimensions:    (time: 120, latitude: 361, longitude: 541)
Coordinates:
  * latitude   (latitude) float32 1kB 20.0 20.08 20.17 ... 49.83 49.92 50.0
  * longitude  (longitude) float32 2kB 115.0 115.1 115.2 ... 159.8 159.9 160.0
  * time       (time) datetime64[ns] 960B 2015-01-01 2015-02-01 ... 2024-12-01
Data variables:
    mlotst     (time, latitude, longitude) float64 187MB dask.array<chunksize=(68, 192, 541), meta=np.ndarray>
Attributes: (12/15)
    Conventions:                   CF-1.6
    area:                          GLOBAL
    contact:                       servicedesk.cmems@mercator-ocean.eu
    credit:                        E.U. Copernicus Marine Service Information...
    dataset:                       global-reanalysis-001-030-monthly
    institution:                   Mercator Ocean
    ...                            ...
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://ma

In [11]:
ds = ds.load()

save_path1 = SAVE_DIR2 / "glorys_mld_2015_2024_kuroshio.nc"

ds.to_netcdf(save_path1, engine='scipy')

print({save_path1})

{WindowsPath('../data/raw/glorys/glorys_mld_2015_2024_kuroshio.nc')}
